# Pre-processing

Loading of the graph generator

In [0]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import subprocess
from time import time
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DataType, FloatType, MapType, Row
import pyspark.sql.functions as F 

In [0]:
dbutils.widgets.text("username", "", "Please enter your git username")
dbutils.widgets.text("password", "", "Please enter your password")

In [0]:
time.sleep(5)

In [0]:
# username = dbutils.widgets.get("username")
# password = dbutils.widgets.get("password")
username = 'sparkistan'
password = 'Nutella336kJ'
url = "https://" + username + ":" + password + "@github.com/MaximeRedstone/graphsMapReduce.git"
email = username + username + "@gmail.com"
dbutils.widgets.removeAll()

In [0]:
!rm -rf graphsMapReduce

In [0]:
!git config --global user.email $email
!git config --global user.name $username
  
if os.path.exists(os.path.join(os.getcwd(), "graphsMapReduce")):
  !cd graphsMapReduce && git pull
else:
  !git clone $url

!cd graphsMapReduce/generator && make re
!mkdir graphsMapReduce/data

Cloning into 'graphsMapReduce'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 0% (1/238)remote: Counting objects: 1% (3/238)remote: Counting objects: 2% (5/238)remote: Counting objects: 3% (8/238)remote: Counting objects: 4% (10/238)remote: Counting objects: 5% (12/238)remote: Counting objects: 6% (15/238)remote: Counting objects: 7% (17/238)remote: Counting objects: 8% (20/238)remote: Counting objects: 9% (22/238)remote: Counting objects: 10% (24/238)remote: Counting objects: 11% (27/238)remote: Counting objects: 12% (29/238)remote: Counting objects: 13% (31/238)remote: Counting objects: 14% (34/238)remote: Counting objects: 15% (36/238)remote: Counting objects: 16% (39/238)remote: Counting objects: 17% (41/238)remote: Counting objects: 18% (43/238)remote: Counting objects: 19% (46/238)remote: Counting objects: 20% (48/238)remote: Counting objects: 21% (50/238)remote: Counting objects: 22% (53/238)remote: Counting objects: 23% (55/238)remote: Counting objects: 24% (58/238)remote: Counting objects: 25% (60/238)remote: Counting objects: 26% (62/238)remote: Counting objects: 27% (65/238)remote: Counting objects: 28% (67/238)remote: Counting objects: 29% (70/238)remote: Counting objects: 30% (72/238)remote: Counting objects: 31% (74/238)remote: Counting objects: 32% (77/238)remote: Counting objects: 33% (79/238)remote: Counting objects: 34% (81/238)remote: Counting objects: 35% (84/238)remote: Counting objects: 36% (86/238)remote: Counting objects: 37% (89/238)remote: Counting objects: 38% (91/238)remote: Counting objects: 39% (93/238)remote: Counting objects: 40% (96/238)remote: Counting objects: 41% (98/238)remote: Counting objects: 42% (100/238)remote: Counting objects: 43% (103/238)remote: Counting objects: 44% (105/238)remote: Counting objects: 45% (108/238)remote: Counting objects: 46% (110/238)remote: Counting objects: 47% (112/238)remote: Counting objects: 48% (115/238)remote: Counting objects: 49% (117/238)remote: Counting objects: 50% (119/238)remote: Counting objects: 51% (122/238)remote: Counting objects: 52% (124/238)remote: Counting objects: 53% (127/238)remote: Counting objects: 54% (129/238)remote: Counting objects: 55% (131/238)remote: Counting objects: 56% (134/238)remote: Counting objects: 57% (136/238)remote: Counting objects: 58% (139/238)remote: Counting objects: 59% (141/238)remote: Counting objects: 60% (143/238)remote: Counting objects: 61% (146/238)remote: Counting objects: 62% (148/238)remote: Counting objects: 63% (150/238)remote: Counting objects: 64% (153/238)remote: Counting objects: 65% (155/238)remote: Counting objects: 66% (158/238)remote: Counting objects: 67% (160/238)remote: Counting objects: 68% (162/238)remote: Counting objects: 69% (165/238)remote: Counting objects: 70% (167/238)remote: Counting objects: 71% (169/238)remote: Counting objects: 72% (172/238)remote: Counting objects: 73% (174/238)remote: Counting objects: 74% (177/238)remote: Counting objects: 75% (179/238)remote: Counting objects: 76% (181/238)remote: Counting objects: 77% (184/238)remote: Counting objects: 78% (186/238)remote: Counting objects: 79% (189/238)remote: Counting objects: 80% (191/238)remote: Counting objects: 81% (193/238)remote: Counting objects: 82% (196/238)remote: Counting objects: 83% (198/238)remote: Counting objects: 84% (200/238)remote: Counting objects: 85% (203/238)remote: Counting objects: 86% (205/238)remote: Counting objects: 87% (208/238)remote: Counting objects: 88% (210/238)remote: Counting objects: 89% (212/238)remote: Counting objects: 90% (215/238)remote: Counting objects: 91% (217/238)remote: Counting objects: 92% (219/238)remote: Counting objects: 93% (222/238)remote: Counting objects: 94% (224/238)remote: Counting objects: 95% (227/238)remote: Counting objects: 96% (229/238)remote: Counting objects: 97% (231/238)remote: Counting objects: 98% (234/238)remote: Counting obj

In [0]:
# os.chdir('..')
# os.getcwd()

In [0]:
user_inputs = {
  '10M': 5,
  '20M': 5,
  '30M': 5,
  '40M': 5,
  '50M': 5,
  '60M': 5,
  '80M': 5
}
#!cd data
# os.chdir('..')
print(os.getcwd())
if os.getcwd().split('/')[-1]=='driver':
  os.chdir('graphsMapReduce')
  
files = os.listdir('data')

print(files)

graph_names = [file.split('_')[1:3] for file in files if "graph" in file]
name_tuples = [(name[0], int(name[1].split('.')[0])) for name in graph_names]
abs_filepathes_names = [os.path.join('data', filename) for filename in files if "graph" in filename]
#filestore_names = [os.path.join('dbfs:/FileStore', 'graphsMapReduce', 'data', filename) for filename in files if "graph" in filename]

abs_filepathes_dict = {key:value for key, value in zip(name_tuples, abs_filepathes_names)}

abs_filepathes_dict['toy'] = os.path.join('data', 'toy_dataset.txt')
abs_filepathes_dict['google'] = os.path.join('data', 'web-Google.txt')

abs_filepathes_list = list(abs_filepathes_dict.values())

for k, v in user_inputs.items():
  
  filename = f'graph_{k}_{v}.txt'
  rel_filepath = os.path.join("..", "data", filename)
#   abs_filepath = os.path.join("data",filename)
  !cd generator && ./graph_generator $k $rel_filepath $v
  abs_filepath = os.path.join('data', filename)
  abs_filepathes_dict[(k,v)] = abs_filepath

filestore_paths_dict = {k:os.path.join('dbfs:/FileStore', 'graphsMapReduce', v) for k,v in abs_filepathes_dict.items()}
print(abs_filepathes_dict.keys())
print(filestore_paths_dict.keys())
#abs_filepathes = [file.replace('dbfs:/FileStore/graphsMapReduce/', '') for file in filestore_paths_dict.values()]

for abs_filepath in abs_filepathes_list:
#   dbutils.fs.rm(filestore_name)
#   f = open(abs_filepath, "r")
  f = open(abs_filepath, "r")
  string = f.read()
  filestore_name = os.path.join("dbfs:/FileStore", 'graphsMapReduce', abs_filepath) 
  print(filestore_name)
#   dbutils.fs.put(filestore_name, string)
  try:
    dbutils.fs.put(filestore_name, string)
  except:
    print(f'Skipped file {filestore_name}')

/databricks/driver
['graph_10M_5.txt', 'graph_15M_11.txt', 'graph_15M_5.txt', 'toy_dataset.txt', 'graph_1M_5.txt', 'graph_10M_11.txt', 'graph_25M_5.txt', 'graph_4M_3.txt', 'graph_45M_5.txt', 'graph_10M_3.txt', 'graph_6M_5.txt', 'graph_10M_7.txt', 'graph_5M_5.txt', 'web-Google.txt']
Start program at Mon Mar 29 18:49:49 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 166950
Final number of edges approximation is 834754
End program at Mon Mar 29 18:49:49 2021
Start program at Mon Mar 29 18:49:50 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 316202
Final number of edges approximation is 1581012
End program at Mon Mar 29 18:49:51 2021
Start program at Mon Mar 29 18:49:52 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 466559
Final number of edges approximation is 2332796
End program at Mon Mar 29 18:49:53 2021
Start program at Mon Mar 29 18:49:54 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 616617
Final number of edges approximation is 3083086
End program at Mon Mar 29 18:49:56 2021
Start program at Mon Mar 29 18:49:57 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 764818
Final number of edges approximation is 3824094
End program at Mon Mar 29 18:49:59 2021
Start program at Mon Mar 29 18:50:00 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 914619
Final number of edges approximation is 4573098
End program at Mon Mar 29 18:50:02 2021
Start program at Mon Mar 29 18:50:03 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 1185489
Final number of edges approximation is 5927445
End program at Mon Mar 29 18:50:07 2021
dict_keys([('10M', 5), ('15M', 11), ('15M', 5), ('1M', 5), ('10M', 11), ('25M', 5), ('4M', 3), ('45M', 5), ('10M', 3), ('6M', 5), ('10M', 7), ('5M', 5), 'toy', 'google', ('20M', 5), ('30M', 5), ('40M', 5), ('50M', 5), ('60M', 5), ('80M', 5)])
dict_keys([('10M', 5), ('15M', 11), ('15M', 5), ('1M', 5), ('10M', 11), ('25M', 5), ('4M', 3), ('45M', 5), ('10M', 3), ('6M', 5), ('10M', 7), ('5M', 5), 'toy', 'google', ('20M', 5), ('30M', 5), ('40M', 5), ('50M', 5), ('60M', 5), ('80M', 5)])
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt
dbfs:/FileStore/graphsMapReduce/data/graph_15M_11.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_15M_11.txt
dbfs:/FileStore/graphsMapReduce/data/graph_15M_5.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_15M_5.txt
dbfs:/FileStore/graphsMapReduce/data/graph_1M_5.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_1M_5.txt
dbfs:/FileStore/graphsMapReduce/data/graph_10M_11.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_10M_11.txt
dbfs:/FileStore/graphsMapReduce/data/graph_25M_5.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_25M_5.txt
dbfs:/FileStore/graphsMapReduce/data/graph_4M_3.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_4M_3.txt
dbfs:/FileStore/graphsMapReduce/data/graph_45M_5.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_45M_5.txt
dbfs:/FileStore/graphsMapReduce/data/graph_10M_3.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_10M_3.txt
dbfs:/FileStore/graphsMapReduce/data/graph_6M_5.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_6M_5.txt
dbfs:/FileStore/graphsMapReduce/data/graph_10M_7.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_10M_7.txt
dbfs:/FileStore/graphsMapReduce/data/graph_5M_5.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/graph_5M_5.txt
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt
Skipped file dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt
dbfs:/FileStore/graphsMapReduce/data/web-Google.txt
Skipped file dbfs:/FileStore/grap

In [0]:
# target = filestore_paths_dict[('60M', 5)]
# print(target)
# dbutils.fs.rm(target)
# k = '80M'
# v = 5
# filename = f'graph_{k}_{v}.txt'
# rel_filepath = os.path.join("..", "data", filename)
# #   abs_filepath = os.path.join("data",filename)
# !cd generator && ./graph_generator $k $rel_filepath $v
# abs_filepath = os.path.join('data', filename)
# f = open(abs_filepath, "r")
# string = f.read()
# filestore_name = os.path.join("dbfs:/FileStore", 'graphsMapReduce', abs_filepath) 
# print(filestore_name)
# #   dbutils.fs.put(filestore_name, string)
# try:
#   dbutils.fs.put(filestore_name, string)
# except:
#   print(f'Skipped file {filestore_name}')


Start program at Sun Mar 28 18:50:09 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 1185489
Final number of edges approximation is 5927445
End program at Sun Mar 28 18:50:12 2021
dbfs:/FileStore/graphsMapReduce/data/graph_80M_5.txt
Wrote 83886255 bytes.

In [0]:
print(filestore_name)
filestore_paths_dict[('80M', 5)] = filestore_name

dbfs:/FileStore/graphsMapReduce/data/graph_80M_5.txt

# RDD implementation

In [0]:
def processGraphRDD(filepath):
  
  def reduce_ccf(x):
    key = x[0]
    values = x[1]
    min_value = values.pop(values.index(min(values)))
    ret = []
    if min_value < key:
      ret.append((key, min_value))
      for value in values:
        accum.add(1) # là on peu surement add en une fois la taille de la liste
        ret.append((value, min_value)) # là on peut surement zip 
    return (ret)
  
  
  text_file = sc.textFile(filepath)
  
  text_file = text_file.filter(lambda x: "#" not in x)
  text_file_split = text_file.map(lambda x: x.split())
  input = text_file_split.map(lambda x: (int(x[0]), int(x[1])))

  accum = sc.accumulator(1)
  while accum.value > 0:

    accum.value = 0
    print("Start loop ", accum.value)

    # CCF-Iterate
    it_map = input.flatMap(lambda x: ((x[0], x[1]), (x[1], x[0])))
#     it_groupby = it_map.groupByKey().mapValues(lambda x: sorted(x))
    it_groupby = it_map.groupByKey().mapValues(list)
    it_reduce = it_groupby.flatMap(lambda x: reduce_ccf(x))
    
    # CCF-Dedup
#     ded_map = it_reduce.map(lambda x: ((x[0], x[1]), None))
#     ded_groupby = ded_map.groupByKey().mapValues(list)
#     input = ded_groupby.map(lambda x: (x[0][0], x[0][1]))
    input = it_reduce.distinct()

    viz = input.collect()
    print('-------------')

  print("Processed file")
#   print(viz)

In [0]:
processGraphRDD(filepath)

In [0]:
!ls graphsMapReduce/data

In [0]:
event_dir = str(subprocess.check_output(['ls', 'eventlogs']))
event_dir = int(event_dir.split("\'")[1][:-2])
full_old_path = os.path.join('eventlogs', str(event_dir), 'eventlog')
full_new_path = os.path.join('eventlogs', str(event_dir), 'eventlog.json')

In [0]:
!cd graphsMapReduce && git pull

In [0]:
comment = ""
!pip install logzero
!mv $full_old_path $full_new_path
!cd graphsMapReduce/src && python3 main.py -e $event_dir #-c $comment

In [0]:
!cd graphsMapReduce/generator && make fclean
!cd graphsMapReduce && git add -A
!cd graphsMapReduce && git config --global user.email "sparkistan@gmail.com"
!cd graphsMapReduce && git config --global user.name "sparkistan"
!cd graphsMapReduce && git commit -m "Added logs"
!cd graphsMapReduce && git push $url

# DataFrame implementation

In [0]:
## CURRENT WORKING VERSION ##

def graphsDF(filestore_name):
  
  def reduce_ccf(key, values):
    min_value = values.pop(values.index(min(values)))
    ret = {}
    ret[key] = min_value
    if min_value < key:
      for value in values:
        acc.add(1)
        ret[value] = min_value
        #ret.append((value, min_value)) # là on peut surement zip 
    else:
      ret = None
    return ret

  reducer = F.udf(lambda x, y: reduce_ccf(x, y), MapType(IntegerType(), IntegerType()))

  schema = StructType([
      StructField("key", IntegerType(), True),
      StructField("value", IntegerType(), True)])

  df = spark.read.format('csv').load(filestore_name, headers=False, delimiter='\t', schema=schema)
  df = df.na.drop()
  
  extracted_val = 1
  acc = sc.accumulator(0)

  while extracted_val != 0:
    previous = acc.value
    # CCF-Iterate
    df_inverter = df.select(F.col('value').alias('key'), F.col('key').alias('value'))
    df = df.union(df_inverter)

      # Creates the (key, [values]) pairs
    df = df.groupBy('key').agg(F.collect_list('value').alias('value'))

    #df = df.groupBy(F.map_keys('key')[0].alias('key')).agg(F.collect_list(F.map_values('key')[0]).alias('value'))
    acc.value = 0
    df = df.withColumn('reducer', reducer('key', 'value')).select('reducer')
    df.collect()
    current = acc.value
    #df.show()
    # Explode & clear nones
    df = df.select(F.explode('reducer'))
    df = df.na.drop()
  #   print('Final output: ')
  #   df.show()

    # CCF - Dedup
    df = df.dropDuplicates()
    #df.show()
    extracted_val = current - previous
    #extracted_val += 1
    print('Final extract :', extracted_val)


  return df

# filestore_paths_dict['toy'] = filestore_name
# filestore_name = filestore_paths_dict[("5M", 5)]
df = graphsDF(filestore_name)
#df.show()

# Logs comparison

In [0]:
filestore_name = filestore_paths_dict['google']

In [0]:
def add_log(df, filepath, algo_type, loop_counter, command, timestamp, accum, length):
    
    single_row = pd.DataFrame(columns=['filename', 'algo_type', 'loop_counter', 'command', 'end_of_command', 'accum', 'length'])
    single_row.loc[0] = [filepath, algo_type, loop_counter, command, timestamp, accum, length]
    with_row = pd.concat([df, single_row])
    return with_row

def processGraphRDDLogs(filepath, df_logs):
  
    def reduce_ccf_min(x):
        key = x[0]
        values = x[1]
        min_value = values.pop(np.argmin(values))
        ret = []
        if min_value < key:
            ret.append((key, min_value))
            accum.add(len(values))
            for value in values:
                ret.append((value, min_value))
        return (ret)

    text_file = sc.textFile(filepath)
    text_file = text_file.filter(lambda x: "#" not in x)
    
    df_logs = add_log(df_logs, filepath, "python-rdd", 0, "start", datetime.now(), 0, 0)
    text_file_split = text_file.map(lambda x: x.split())
    input = text_file_split.map(lambda x: (int(x[0]), int(x[1])))

    accum = sc.accumulator(1)
    loop_counter = 1
    while accum.value != 0:

        accum.value = 0
        print(f"----------\nStart loop at {datetime.now()}, accum_value is {accum.value}")

        # CCF-Iterate
        it_map = input.flatMap(lambda x: ((x[0], x[1]), (x[1], x[0])))
#         it_map.collect()
#         df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "it_map", datetime.now(), accum.value)
        
        it_groupby = it_map.groupByKey().mapValues(list)
#         it_groupby.collect()
#         df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "it_groupby", datetime.now(), accum.value)
        
        it_reduce = it_groupby.flatMap(lambda x: reduce_ccf_min(x))
#         it_reduce.collect()
#         df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "it_reduce", datetime.now(), accum.value)
        
        
#         df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "dedup", datetime.now(), accum.value)

        # CCF-Dedup
  
        input = it_reduce.distinct()
        ret = input.collect()
          
          
#         ded_map = it_reduce.map(lambda x: ((x[0], x[1]), None))
#         ded_map.collect()
#         df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "ded_map", datetime.now(), accum.value)
        
#         ded_groupby = ded_map.groupByKey().mapValues(list)
#         ded_groupby.collect()
#         df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "ded_groupby", datetime.now(), accum.value)
        
#         input = ded_groupby.map(lambda x: (x[0][0], x[0][1]))        
#         viz = input.collect()

        df_logs = add_log(df_logs, filepath, "python-rdd", loop_counter, "ded_reduce", datetime.now(), accum.value, len(ret))

        print(f"End loop at {datetime.now()}, accum_value is {accum.value}")
        loop_counter += 1

    print(f"----------\nProcessed file at {datetime.now()}\n----------")
    return df_logs

def processGraphDFLogsV2(filepath, df_logs):
  
  def reduce_ccf(key, values):
    min_value = values.pop(values.index(min(values)))
    ret = {}
    ret[key] = min_value
    if min_value < key:
      for value in values:
        acc.add(1)
        ret[value] = min_value
    else:
      ret = None
    return ret

  reducer = F.udf(lambda x, y: reduce_ccf(x, y), MapType(IntegerType(), IntegerType())).asNondeterministic()

  schema = StructType([
      StructField("key", IntegerType(), True),
      StructField("value", IntegerType(), True)])

  df = spark.read.format('csv').load(filepath, headers=False, delimiter='\t', schema=schema)
  df = df.na.drop()
  df_logs = add_log(df_logs, filepath, "python-df", 0, "start", datetime.now(), 0, 0)

  acc = sc.accumulator(1)
  loop_counter = 1
  
  while acc.value != 0:
    
    acc.value = 0
    print(f"----------\nStart loop at {datetime.now()}, accumulator value is {acc.value}")
    
    # CCF-Iterate
    df_inverter = df.select(F.col('value').alias('key'), F.col('key').alias('value'))
    df = df.union(df_inverter)
    
#     if logs==True:
#       acc.value = 0
#       debug = df.collect()
#       df_logs_ret = add_log(df_logs, filepath, "python-df", loop_counter, "it-map", datetime.now(), acc.value, len(debug)) 

    df = df.groupBy('key').agg(F.collect_list('value').alias('value'))
    df = df.withColumn('reducer', reducer('key', 'value')).select('reducer')
    df = df.select(F.explode('reducer'))
    df = df.na.drop()
    
    # CCF - Dedup
    df = df.distinct()
    collected = df.collect()
    df = spark.createDataFrame(sc.parallelize(collected), schema)
    print(f"End loop at {datetime.now()}, final value is {acc.value}")
    df_logs = add_log(df_logs, filepath, "python-df", loop_counter, "ded_reduce", datetime.now(), acc.value, len(collected))
    loop_counter += 1
  
  print(f"----------\nProcessed file at {datetime.now()}\n----------")
  return df_logs

In [0]:
df_logs = pd.DataFrame(columns=['filename', 'algo_type', 'loop_counter', 'command', 'end_of_command', 'accum', 'length'])
#filestore_name = filestore_paths_dict[("60M", 5)]

In [0]:
filestore_name = filestore_paths_dict['toy']
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_toy = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_toy)

----------
Start loop at 2021-03-28 14:09:34.782933, accum_value is 0
End loop at 2021-03-28 14:09:35.630134, accum_value is 4
----------
Start loop at 2021-03-28 14:09:35.630192, accum_value is 0
End loop at 2021-03-28 14:09:35.989464, accum_value is 9
----------
Start loop at 2021-03-28 14:09:35.989895, accum_value is 0
End loop at 2021-03-28 14:09:36.331238, accum_value is 4
----------
Start loop at 2021-03-28 14:09:36.331298, accum_value is 0
End loop at 2021-03-28 14:09:36.664314, accum_value is 0
----------
Processed file at 2021-03-28 14:09:36.664665
----------
----------
Start loop at 2021-03-28 14:09:36.983719, accumulator value is 0
End loop at 2021-03-28 14:09:38.462247, final value is 4
----------
Start loop at 2021-03-28 14:09:38.481055, accumulator value is 0
End loop at 2021-03-28 14:09:39.587342, final value is 9
----------
Start loop at 2021-03-28 14:09:39.606334, accumulator value is 0
End loop at 2021-03-28 14:09:40.617729, final value is 4
----------
Start loop at 2021-03-28 14:09:40.633493, accumulator value is 0
End loop at 2021-03-28 14:09:41.773061, final value is 0
----------
Processed file at 2021-03-28 14:09:41.800545
----------

filename,algo_type,loop_counter,command,end_of_command,accum,length
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-rdd,0,start,2021-03-28T14:09:34.758+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-rdd,1,ded_reduce,2021-03-28T14:09:35.610+0000,4,10
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-rdd,2,ded_reduce,2021-03-28T14:09:35.968+0000,9,8
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-rdd,3,ded_reduce,2021-03-28T14:09:36.304+0000,4,6
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-rdd,4,ded_reduce,2021-03-28T14:09:36.645+0000,0,6
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-df,0,start,2021-03-28T14:09:36.972+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-df,1,ded_reduce,2021-03-28T14:09:38.462+0000,4,10
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-df,2,ded_reduce,2021-03-28T14:09:39.587+0000,9,8
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-df,3,ded_reduce,2021-03-28T14:09:40.617+0000,4,6
dbfs:/FileStore/graphsMapReduce/data/toy_dataset.txt,python-df,4,ded_reduce,2021-03-28T14:09:41.773+0000,0,6


In [0]:
filestore_name = filestore_paths_dict['google']
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_google = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_google)

In [0]:
filestore_name = filestore_paths_dict[('10M', 5)]
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_10 = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_10)

----------
Start loop at 2021-03-28 14:09:43.431773, accum_value is 0
End loop at 2021-03-28 14:10:08.683395, accum_value is 1355603
----------
Start loop at 2021-03-28 14:10:08.683785, accum_value is 0
End loop at 2021-03-28 14:10:47.923675, accum_value is 2683303
----------
Start loop at 2021-03-28 14:10:47.923733, accum_value is 0
End loop at 2021-03-28 14:11:38.311997, accum_value is 5305417
----------
Start loop at 2021-03-28 14:11:38.312063, accum_value is 0
End loop at 2021-03-28 14:12:02.720050, accum_value is 5251057
----------
Start loop at 2021-03-28 14:12:02.720116, accum_value is 0
End loop at 2021-03-28 14:12:08.596630, accum_value is 129530
----------
Start loop at 2021-03-28 14:12:08.596693, accum_value is 0
End loop at 2021-03-28 14:12:13.589412, accum_value is 0
----------
Processed file at 2021-03-28 14:12:13.589468
----------
----------
Start loop at 2021-03-28 14:12:13.967871, accumulator value is 0
End loop at 2021-03-28 14:12:33.009645, final value is 1355603
----------
Start loop at 2021-03-28 14:12:33.020748, accumulator value is 0
End loop at 2021-03-28 14:13:27.358485, final value is 2683303
----------
Start loop at 2021-03-28 14:13:27.369533, accumulator value is 0
End loop at 2021-03-28 14:14:47.900604, final value is 5305417
----------
Start loop at 2021-03-28 14:14:47.912275, accumulator value is 0
End loop at 2021-03-28 14:15:53.389628, final value is 5251057
----------
Start loop at 2021-03-28 14:15:53.401164, accumulator value is 0
End loop at 2021-03-28 14:16:01.295048, final value is 129530
----------
Start loop at 2021-03-28 14:16:01.306138, accumulator value is 0
End loop at 2021-03-28 14:16:07.652331, final value is 0
----------
Processed file at 2021-03-28 14:16:07.664139
----------

filename,algo_type,loop_counter,command,end_of_command,accum,length
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-rdd,0,start,2021-03-28T14:09:43.419+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-rdd,1,ded_reduce,2021-03-28T14:10:08.669+0000,1355603,1505541
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-rdd,2,ded_reduce,2021-03-28T14:10:47.911+0000,2683303,2749376
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-rdd,3,ded_reduce,2021-03-28T14:11:38.295+0000,5305417,2763269
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-rdd,4,ded_reduce,2021-03-28T14:12:02.708+0000,5251057,231707
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-rdd,5,ded_reduce,2021-03-28T14:12:08.585+0000,129530,166942
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-rdd,6,ded_reduce,2021-03-28T14:12:13.577+0000,0,166942
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-df,0,start,2021-03-28T14:12:13.953+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-df,1,ded_reduce,2021-03-28T14:12:33.009+0000,1355603,1505541
dbfs:/FileStore/graphsMapReduce/data/graph_10M_5.txt,python-df,2,ded_reduce,2021-03-28T14:13:27.358+0000,2683303,2749376


In [0]:
filestore_name = filestore_paths_dict[('20M', 5)]
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_20 = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_20)

----------
Start loop at 2021-03-28 14:16:08.161439, accum_value is 0
End loop at 2021-03-28 14:16:55.195421, accum_value is 2585495
----------
Start loop at 2021-03-28 14:16:55.195480, accum_value is 0
End loop at 2021-03-28 14:20:39.902159, accum_value is 5123719
----------
Start loop at 2021-03-28 14:20:39.902217, accum_value is 0
End loop at 2021-03-28 14:26:25.415308, accum_value is 10157224
----------
Start loop at 2021-03-28 14:26:25.415369, accum_value is 0
End loop at 2021-03-28 14:28:46.180209, accum_value is 11726632
----------
Start loop at 2021-03-28 14:28:46.180567, accum_value is 0
End loop at 2021-03-28 14:28:57.839453, accum_value is 315564
----------
Start loop at 2021-03-28 14:28:57.839824, accum_value is 0
End loop at 2021-03-28 14:29:07.239325, accum_value is 0
----------
Processed file at 2021-03-28 14:29:07.239383
----------
----------
Start loop at 2021-03-28 14:29:07.601551, accumulator value is 0
End loop at 2021-03-28 14:29:46.259515, final value is 2585495
----------
Start loop at 2021-03-28 14:29:46.274150, accumulator value is 0
End loop at 2021-03-28 14:31:32.006193, final value is 5123719
----------
Start loop at 2021-03-28 14:31:32.017324, accumulator value is 0
End loop at 2021-03-28 14:34:12.956838, final value is 10157224
----------
Start loop at 2021-03-28 14:34:12.970029, accumulator value is 0
End loop at 2021-03-28 14:36:38.676960, final value is 11726632
----------
Start loop at 2021-03-28 14:36:38.690326, accumulator value is 0
End loop at 2021-03-28 14:36:55.379045, final value is 315564
----------
Start loop at 2021-03-28 14:36:55.390553, accumulator value is 0
End loop at 2021-03-28 14:37:07.265445, final value is 0
----------
Processed file at 2021-03-28 14:37:07.276568
----------

filename,algo_type,loop_counter,command,end_of_command,accum,length
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-rdd,0,start,2021-03-28T14:16:08.150+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-rdd,1,ded_reduce,2021-03-28T14:16:55.184+0000,2585495,2869743
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-rdd,2,ded_reduce,2021-03-28T14:20:39.889+0000,5123719,5262992
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-rdd,3,ded_reduce,2021-03-28T14:26:25.403+0000,10157224,6108855
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-rdd,4,ded_reduce,2021-03-28T14:28:46.168+0000,11726632,473962
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-rdd,5,ded_reduce,2021-03-28T14:28:57.818+0000,315564,316180
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-rdd,6,ded_reduce,2021-03-28T14:29:07.227+0000,0,316180
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-df,0,start,2021-03-28T14:29:07.589+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-df,1,ded_reduce,2021-03-28T14:29:46.259+0000,2585495,2869743
dbfs:/FileStore/graphsMapReduce/data/graph_20M_5.txt,python-df,2,ded_reduce,2021-03-28T14:31:32.006+0000,5123719,5262992


In [0]:
filestore_name = filestore_paths_dict[('30M', 5)]
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_30 = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_30)

----------
Start loop at 2021-03-28 14:37:07.640948, accum_value is 0
End loop at 2021-03-28 14:38:53.605903, accum_value is 3813673
----------
Start loop at 2021-03-28 14:38:53.605957, accum_value is 0
End loop at 2021-03-28 14:43:42.180276, accum_value is 7545282
----------
Start loop at 2021-03-28 14:43:42.180335, accum_value is 0
End loop at 2021-03-28 14:50:42.597563, accum_value is 14974496
----------
Start loop at 2021-03-28 14:50:42.597620, accum_value is 0
End loop at 2021-03-28 14:53:53.056551, accum_value is 19324891
----------
Start loop at 2021-03-28 14:53:53.056918, accum_value is 0
End loop at 2021-03-28 14:54:11.533389, accum_value is 750098
----------
Start loop at 2021-03-28 14:54:11.533451, accum_value is 0
End loop at 2021-03-28 14:54:25.136066, accum_value is 0
----------
Processed file at 2021-03-28 14:54:25.136120
----------
----------
Start loop at 2021-03-28 14:54:25.492577, accumulator value is 0
End loop at 2021-03-28 14:55:15.623081, final value is 3813673
----------
Start loop at 2021-03-28 14:55:15.634893, accumulator value is 0
End loop at 2021-03-28 14:57:42.516499, final value is 7545282
----------
Start loop at 2021-03-28 14:57:42.614366, accumulator value is 0
End loop at 2021-03-28 15:01:52.739025, final value is 14974496
----------
Start loop at 2021-03-28 15:01:52.822815, accumulator value is 0
End loop at 2021-03-28 15:06:08.192703, final value is 19324891
----------
Start loop at 2021-03-28 15:06:08.424410, accumulator value is 0
End loop at 2021-03-28 15:06:34.900753, final value is 750098
----------
Start loop at 2021-03-28 15:06:34.916000, accumulator value is 0
End loop at 2021-03-28 15:06:53.725821, final value is 0
----------
Processed file at 2021-03-28 15:06:53.738006
----------

filename,algo_type,loop_counter,command,end_of_command,accum,length
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-rdd,0,start,2021-03-28T14:37:07.629+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-rdd,1,ded_reduce,2021-03-28T14:38:53.594+0000,3813673,4233161
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-rdd,2,ded_reduce,2021-03-28T14:43:42.168+0000,7545282,7760631
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-rdd,3,ded_reduce,2021-03-28T14:50:42.583+0000,14974496,9960879
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-rdd,4,ded_reduce,2021-03-28T14:53:53.043+0000,19324891,841589
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-rdd,5,ded_reduce,2021-03-28T14:54:11.519+0000,750098,466540
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-rdd,6,ded_reduce,2021-03-28T14:54:25.124+0000,0,466540
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-df,0,start,2021-03-28T14:54:25.474+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-df,1,ded_reduce,2021-03-28T14:55:15.623+0000,3813673,4233161
dbfs:/FileStore/graphsMapReduce/data/graph_30M_5.txt,python-df,2,ded_reduce,2021-03-28T14:57:42.516+0000,7545282,7760631


In [0]:
filestore_name = filestore_paths_dict[('40M', 5)]
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_40 = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_40)

----------
Start loop at 2021-03-28 15:06:54.413917, accum_value is 0
End loop at 2021-03-28 15:11:05.662915, accum_value is 5039689
----------
Start loop at 2021-03-28 15:11:05.662979, accum_value is 0
End loop at 2021-03-28 15:17:37.754092, accum_value is 9969585
----------
Start loop at 2021-03-28 15:17:37.754160, accum_value is 0
End loop at 2021-03-28 15:26:33.540209, accum_value is 19791554
----------
Start loop at 2021-03-28 15:26:33.540269, accum_value is 0
End loop at 2021-03-28 15:31:27.858031, accum_value is 27302057
----------
Start loop at 2021-03-28 15:31:27.858916, accum_value is 0
End loop at 2021-03-28 15:31:52.405178, accum_value is 1033908
----------
Start loop at 2021-03-28 15:31:52.405234, accum_value is 0
End loop at 2021-03-28 15:32:11.189655, accum_value is 0
----------
Processed file at 2021-03-28 15:32:11.189712
----------
----------
Start loop at 2021-03-28 15:32:11.623047, accumulator value is 0
End loop at 2021-03-28 15:33:18.070516, final value is 5039689
----------
Start loop at 2021-03-28 15:33:18.082436, accumulator value is 0
End loop at 2021-03-28 15:36:43.712862, final value is 9969585
----------
Start loop at 2021-03-28 15:36:43.802608, accumulator value is 0
End loop at 2021-03-28 15:43:16.744839, final value is 19791554
----------
Start loop at 2021-03-28 15:43:16.845087, accumulator value is 0
End loop at 2021-03-28 15:49:23.191020, final value is 27302057
----------
Start loop at 2021-03-28 15:49:23.445409, accumulator value is 0
End loop at 2021-03-28 15:49:59.969525, final value is 1033908
----------
Start loop at 2021-03-28 15:49:59.981885, accumulator value is 0
End loop at 2021-03-28 15:50:21.547336, final value is 0
----------
Processed file at 2021-03-28 15:50:21.559963
----------

filename,algo_type,loop_counter,command,end_of_command,accum,length
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-rdd,0,start,2021-03-28T15:06:54.402+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-rdd,1,ded_reduce,2021-03-28T15:11:05.651+0000,5039689,5594068
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-rdd,2,ded_reduce,2021-03-28T15:17:37.626+0000,9969585,10259620
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-rdd,3,ded_reduce,2021-03-28T15:26:33.432+0000,19791554,14040899
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-rdd,4,ded_reduce,2021-03-28T15:31:27.843+0000,27302057,1133527
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-rdd,5,ded_reduce,2021-03-28T15:31:52.393+0000,1033908,616573
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-rdd,6,ded_reduce,2021-03-28T15:32:11.177+0000,0,616573
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-df,0,start,2021-03-28T15:32:11.611+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-df,1,ded_reduce,2021-03-28T15:33:18.070+0000,5039689,5594068
dbfs:/FileStore/graphsMapReduce/data/graph_40M_5.txt,python-df,2,ded_reduce,2021-03-28T15:36:43.712+0000,9969585,10259620


In [0]:
filestore_name = filestore_paths_dict[('50M', 5)]
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_50 = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_50)

----------
Start loop at 2021-03-28 15:50:22.346463, accum_value is 0
End loop at 2021-03-28 15:54:54.421192, accum_value is 6270740
----------
Start loop at 2021-03-28 15:54:54.421266, accum_value is 0
End loop at 2021-03-28 16:02:53.874544, accum_value is 12411515
----------
Start loop at 2021-03-28 16:02:53.874618, accum_value is 0
End loop at 2021-03-28 16:15:23.485734, accum_value is 24625387
----------
Start loop at 2021-03-28 16:15:23.485796, accum_value is 0
End loop at 2021-03-28 16:21:22.214733, accum_value is 35354534
----------
Start loop at 2021-03-28 16:21:22.214816, accum_value is 0
End loop at 2021-03-28 16:21:55.387379, accum_value is 1288374
----------
Start loop at 2021-03-28 16:21:55.387442, accum_value is 0
End loop at 2021-03-28 16:22:18.722120, accum_value is 0
----------
Processed file at 2021-03-28 16:22:18.722183
----------
----------
Start loop at 2021-03-28 16:22:19.122225, accumulator value is 0
End loop at 2021-03-28 16:23:40.171246, final value is 6270740
----------
Start loop at 2021-03-28 16:23:40.187369, accumulator value is 0
End loop at 2021-03-28 16:27:52.200736, final value is 12411515
----------
Start loop at 2021-03-28 16:27:52.305367, accumulator value is 0
End loop at 2021-03-28 16:36:43.323097, final value is 24625387
----------
Start loop at 2021-03-28 16:36:43.436528, accumulator value is 0
End loop at 2021-03-28 16:46:16.656289, final value is 35354534
----------
Start loop at 2021-03-28 16:46:16.780089, accumulator value is 0
End loop at 2021-03-28 16:47:03.360323, final value is 1288374
----------
Start loop at 2021-03-28 16:47:03.372040, accumulator value is 0
End loop at 2021-03-28 16:47:30.541378, final value is 0
----------
Processed file at 2021-03-28 16:47:30.552829
----------

filename,algo_type,loop_counter,command,end_of_command,accum,length
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-rdd,0,start,2021-03-28T15:50:22.334+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-rdd,1,ded_reduce,2021-03-28T15:54:54.410+0000,6270740,6958730
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-rdd,2,ded_reduce,2021-03-28T16:02:53.784+0000,12411515,12776038
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-rdd,3,ded_reduce,2021-03-28T16:15:23.365+0000,24625387,18163103
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-rdd,4,ded_reduce,2021-03-28T16:21:22.198+0000,35354534,1408975
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-rdd,5,ded_reduce,2021-03-28T16:21:55.374+0000,1288374,764788
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-rdd,6,ded_reduce,2021-03-28T16:22:18.710+0000,0,764788
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-df,0,start,2021-03-28T16:22:19.109+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-df,1,ded_reduce,2021-03-28T16:23:40.171+0000,6270740,6958730
dbfs:/FileStore/graphsMapReduce/data/graph_50M_5.txt,python-df,2,ded_reduce,2021-03-28T16:27:52.200+0000,12411515,12776038


In [0]:
filestore_name = filestore_paths_dict[('60M', 5)]
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_60 = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_60)

----------
Start loop at 2021-03-28 17:00:51.064526, accum_value is 0
End loop at 2021-03-28 17:05:53.761252, accum_value is 7498162
----------
Start loop at 2021-03-28 17:05:53.761309, accum_value is 0
End loop at 2021-03-28 17:14:51.454977, accum_value is 14845087
----------
Start loop at 2021-03-28 17:14:51.455035, accum_value is 0
End loop at 2021-03-28 17:29:18.220752, accum_value is 29491643
----------
Start loop at 2021-03-28 17:29:18.220810, accum_value is 0
End loop at 2021-03-28 17:36:22.469455, accum_value is 43792820
----------
Start loop at 2021-03-28 17:36:22.469863, accum_value is 0
End loop at 2021-03-28 17:36:59.291756, accum_value is 1758802
----------
Start loop at 2021-03-28 17:36:59.291950, accum_value is 0
End loop at 2021-03-28 17:37:26.163685, accum_value is 0
----------
Processed file at 2021-03-28 17:37:26.163742
----------
----------
Start loop at 2021-03-28 17:37:26.586522, accumulator value is 0
End loop at 2021-03-28 17:39:06.178094, final value is 7498162
----------
Start loop at 2021-03-28 17:39:06.189274, accumulator value is 0
End loop at 2021-03-28 17:45:35.761981, final value is 14845087
----------
Start loop at 2021-03-28 17:45:35.858270, accumulator value is 0
End loop at 2021-03-28 18:04:59.972026, final value is 29491643
----------
Start loop at 2021-03-28 18:05:00.143058, accumulator value is 0
End loop at 2021-03-28 18:15:44.829484, final value is 43792820
----------
Start loop at 2021-03-28 18:15:44.994106, accumulator value is 0
End loop at 2021-03-28 18:16:39.868200, final value is 1758802
----------
Start loop at 2021-03-28 18:16:39.884939, accumulator value is 0
End loop at 2021-03-28 18:17:10.825329, final value is 0
----------
Processed file at 2021-03-28 18:17:10.840369
----------

filename,algo_type,loop_counter,command,end_of_command,accum,length
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-rdd,0,start,2021-03-28T17:00:51.051+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-rdd,1,ded_reduce,2021-03-28T17:05:53.750+0000,7498162,8320726
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-rdd,2,ded_reduce,2021-03-28T17:14:51.330+0000,14845087,15285983
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-rdd,3,ded_reduce,2021-03-28T17:29:18.131+0000,29491643,22442264
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-rdd,4,ded_reduce,2021-03-28T17:36:22.456+0000,43792820,1793973
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-rdd,5,ded_reduce,2021-03-28T17:36:59.280+0000,1758802,914572
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-rdd,6,ded_reduce,2021-03-28T17:37:26.152+0000,0,914572
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-df,0,start,2021-03-28T17:37:26.566+0000,0,0
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-df,1,ded_reduce,2021-03-28T17:39:06.178+0000,7498162,8320726
dbfs:/FileStore/graphsMapReduce/data/graph_60M_5.txt,python-df,2,ded_reduce,2021-03-28T17:45:35.762+0000,14845087,15285983


In [0]:
filestore_name = filestore_paths_dict[('80M', 5)]
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)
df_logs_80 = processGraphDFLogsV2(filestore_name, df_logs_rdd)
display(df_logs_80)

----------
Start loop at 2021-03-29 18:50:34.242043, accum_value is 0
End loop at 2021-03-29 18:56:26.802335, accum_value is 9743484
----------
Start loop at 2021-03-29 18:56:26.802413, accum_value is 0